# portfolio optimization (real world)

In [1]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/74/46/e81907704ab203206769dee1385dc77e1407576ff8f50a0681d0a6b541be/colour-0.1.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a1/59/2613468ffbbe3a818934d06b81b9f4877fe054afbf4f99d2f43f398a0b34/cvxpy-1.0.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/bf/2d/005e45738ab07a26e621c9c12dc97381f372e06678adf7dc3356a69b5960/numpy-1.13.3.zip
  Using cached https://files.pythonhosted.org/packages/4c/7e/96aaf955f2d8a829b3ca637368c726b5030bac01f22b6e66da3da361f467/pandas-0.21.1.tar.gz
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/manwaiwong/anaconda2/envs/twd/bin/python /Users/manwaiwong/anaconda2/envs/twd/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/_5/scl16mzs3cj0yy506jvqf6100000gn/T/pip-build-env-ai3hwe8a/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -

ERROR: Command errored out with exit status 1: /Users/manwaiwong/anaconda2/envs/twd/bin/python /Users/manwaiwong/anaconda2/envs/twd/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/_5/scl16mzs3cj0yy506jvqf6100000gn/T/pip-build-env-ai3hwe8a/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- wheel setuptools Cython 'numpy==1.9.3; python_version=='"'"'3.5'"'"'' 'numpy==1.12.1; python_version=='"'"'3.6'"'"'' 'numpy==1.13.1; python_version>='"'"'3.7'"'"'' Check the logs for full command output.


In [3]:
import pandas as pd
import numpy as np

import helper
import project_helper
import os
import pandas_datareader.data as pdr
import datetime as dt
import requests_cache
from alpha_vantage.timeseries import TimeSeries


In [44]:
!pip install cvxpy

In [4]:
# quandl key
f = open("/Users/manwaiwong/key/quandl.txt","r")

if f.mode == 'r':
    c = f.read() 

# get stock stock from quandl

In [5]:
# quandl 
tickers = ["HKEX/00388","HKEX/01810","HKEX/02342", "HKEX/00552"]
expire_after = dt.timedelta(days=1)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
start = dt.date.today() - dt.timedelta(1095)
end = dt.date.today() 
qkey = c


close = pd.DataFrame() # dataframe to store close price of each ticker
volumes = pd.DataFrame()
dividends = pd.DataFrame()
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
## loop tickers get data
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            temp = pdr.DataReader(tickers[i], 'quandl', start, end, api_key=qkey, session=session)
            #print(start)
            #print(end)
            #print(temp)
            close[tickers[i]] = temp["PreviousClose"]
            volumes[tickers[i]] = temp["ShareVolume000"]
            dividends[tickers[i]] = temp["PEx"].fillna(0)
            #temp.dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1




In [6]:
close

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-17,269.6,13.10,3.08,5.88
2020-02-14,268.8,13.38,3.11,5.84
2020-02-13,270.4,12.92,3.11,5.93
2020-02-12,270.2,12.78,3.12,5.80
2020-02-11,268.4,12.96,3.16,5.82
...,...,...,...,...
2017-02-24,200.4,NaN,1.49,5.12
2017-02-23,200.0,NaN,1.49,5.22
2017-02-22,199.0,NaN,1.47,4.95


# generate_dollar_volume_weights

In [7]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    #us = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
    #TODO: Implement function
    #print(close)
    #print(volume.index)
    dollar_volume = close * volume
    dollar_volume_weight = dollar_volume.divide(dollar_volume.sum(axis = 1), axis = 'index' )
    #print(dollar_volume)
    #print('end')
    return dollar_volume_weight


#project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

In [8]:
index_weights = generate_dollar_volume_weights(close, volumes)
index_weights

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-17,0.239781,0.647400,0.089060,0.023760
2020-02-14,0.158773,0.802495,0.021726,0.017006
2020-02-13,0.155104,0.813488,0.017476,0.013932
2020-02-12,0.217015,0.713147,0.041184,0.028653
2020-02-11,0.268362,0.668699,0.035476,0.027462
...,...,...,...,...
2017-02-24,0.944122,NaN,0.004567,0.051311
2017-02-23,0.942744,NaN,0.004700,0.052557
2017-02-22,0.942249,NaN,0.003381,0.054369


# calculate_dividend_weights

In [9]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    dividend_yield_over_time = np.cumsum(dividends, axis = 0)
    dividend_weights = dividend_yield_over_time.divide(dividend_yield_over_time.sum(axis = 1), axis = 'index' )
    return dividend_weights



In [10]:
etf_weights = calculate_dividend_weights(dividends)
etf_weights

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-17,NaN,NaN,NaN,NaN
2020-02-14,NaN,NaN,NaN,NaN
2020-02-13,NaN,NaN,NaN,NaN
2020-02-12,NaN,NaN,NaN,NaN
2020-02-11,NaN,NaN,NaN,NaN
...,...,...,...,...
2017-02-24,0.588510,NaN,0.259147,0.152343
2017-02-23,0.588090,NaN,0.259281,0.152628
2017-02-22,0.587661,NaN,0.259410,0.152929


# Returns

In [11]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    #print(prices.sum(axis = 0))
    # lesson 7 stock return
    returns = (prices - prices.shift(1)) / prices.shift(1)
    return returns

In [12]:
returns = generate_returns(close)
returns

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-17,NaN,NaN,NaN,NaN
2020-02-14,-0.002967,0.021374,0.009740,-0.006803
2020-02-13,0.005952,-0.034380,0.000000,0.015411
2020-02-12,-0.000740,-0.010836,0.003215,-0.021922
2020-02-11,-0.006662,0.014085,0.012821,0.003448
...,...,...,...,...
2017-02-24,0.012121,NaN,0.000000,-0.021033
2017-02-23,-0.001996,NaN,0.000000,0.019531
2017-02-22,-0.005000,NaN,-0.013423,-0.051724


# Weighted Retuen

In [13]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    #print(returns)
    #print(weights)
    #print(returns * weights)
    # lesson 17 quiz 16
    #TODO: Implement function
    weighted_returns = returns * weights
    return weighted_returns



In [14]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
index_weighted_returns

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
Date,,,,
2020-02-17,NaN,NaN,NaN,NaN
2020-02-14,-0.000471,0.017153,0.000212,-0.000116
2020-02-13,0.000923,-0.027967,0.000000,0.000215
2020-02-12,-0.000161,-0.007728,0.000132,-0.000628
2020-02-11,-0.001788,0.009418,0.000455,0.000095
...,...,...,...,...
2017-02-24,0.011444,NaN,0.000000,-0.001079
2017-02-23,-0.001882,NaN,0.000000,0.001026
2017-02-22,-0.004711,NaN,-0.000045,-0.002812


# Cumulative Returns

In [15]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #print(returns)
    #TODO: Implement function
    #https://www.oreilly.com/library/view/learning-pandas-/9781787123137/3105cd10-4928-4219-8444-25a502b46c60.xhtml
    cumulative_returns = (1 + returns.sum(axis = 1)).cumprod(axis = 0)
    return cumulative_returns

In [16]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)

In [17]:
index_weighted_cumulative_returns

Date
2020-02-17    1.000000
2020-02-14    1.016777
2020-02-13    0.989498
2020-02-12    0.981202
2020-02-11    0.989228
                ...   
2017-02-24    0.643785
2017-02-23    0.643234
2017-02-22    0.638366
2017-02-21    0.644191
2017-02-20    0.642218
Length: 735, dtype: float64

# Tracking Error

In [18]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    tracking_error = np.sqrt(252) * (benchmark_returns_by_date - etf_returns_by_date).std()
    #TODO: Implement function

    return tracking_error

In [19]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.2854687566913326


# Portfolio Optimization


$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [20]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #print(returns)
    #TODO: Implement function
    returns_covariance = np.cov(returns.fillna(0).T)
    #print( np.asarray(returns_covariance))
    #print(returns.fillna(0))
    #print(returns.fillna(0).T)
    return returns_covariance

In [21]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)


In [22]:
covariance_returns_correlation

,HKEX/00388,HKEX/01810,HKEX/02342,HKEX/00552
HKEX/00388,1.000000,0.350688,0.344719,0.272976
HKEX/01810,0.350688,1.000000,0.216390,0.176347
HKEX/02342,0.344719,0.216390,1.000000,0.378036
HKEX/00552,0.272976,0.176347,0.378036,1.000000


# optimization cvxpy

In [23]:
def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    
    #print(covariance_returns)
    #TODO: Implement function
    m = covariance_returns.shape[0]
        
    #covariance matrix of returns
    cov = covariance_returns
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x,cov)
    
    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm(x - index_weights)
    
    #objective function
    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)
    
    #constraints
    constraints = [x >= 0 , sum(x) ==1 ]

    #use cvxpy to solve the objective
    cvx.Problem(objective, constraints).solve()
    #retrieve the weights of the optimized portfolio
    #x_values
    x_values = x.value
    
    return x_values

In [24]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

NameError: name 'cvx' is not defined

In [25]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

NameError: name 'optimal_single_rebalance_etf_weights' is not defined

In [24]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    #print(chunk_size)
    #TODO: Implement function
    all_rebalance_weights = []
    #print(range(chunk_size, len(returns)+1, shift_size))
    for i in range(chunk_size, len(returns)+1, shift_size):
        covariance_returns = get_covariance_returns(returns.iloc[i-chunk_size:i])
        optimal_weights = get_optimal_weights(covariance_returns, index_weights.iloc[i-1])
        all_rebalance_weights.append(optimal_weights)
        
    return all_rebalance_weights



In [117]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [30]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    #https://knowledge.udacity.com/questions/65808
    assert shift_size > 0
    assert rebalance_count > 0
    #print(all_rebalance_weights)
    #TODO: Implement function
    sum_total_turnover = np.sum(abs(np.diff(all_rebalance_weights, axis=0)))  
    portfolio_turnover = (sum_total_turnover/rebalance_count)*n_trading_days_in_year / shift_size
    
    return portfolio_turnover



In [31]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

NameError: name 'all_rebalance_weights' is not defined

In [6]:
pk = np.arange(0,10,1).reshape(2,5)